In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import time
import functions

In [ ]:
filepath ='data/'
csvfilenames = ['confinement_training.csv', 
                'member_information.csv', 
                'labresults_training.csv', 
                'medical_training.csv', 
                'rx_training.csv',
                'medical_target.csv',
                'rx_target.csv']

with open('members.csv') as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

members = pd.DataFrame(data)
members.columns = members.iloc[0]
members = members[1:]
members = members.set_index(['PATID']) #makes PATID the row index instead of integers, not sure it's worth it.

with open(filepath+csvfilenames[2]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

labdata = pd.DataFrame(data)
labdata.columns = labdata.iloc[0]
labdata = labdata[1:]

with open('trainingmedicalwanted.csv') as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

medical_wanted = pd.DataFrame(data)
medical_wanted.columns = medical_wanted.iloc[0]
medical_wanted = medical_wanted[1:]

with open(filepath+csvfilenames[4]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

rxdata = pd.DataFrame(data)
rxdata.columns = rxdata.iloc[0]
rxdata = rxdata[1:]

with open('targetmedicalwanted.csv') as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

medical_wanted_t = pd.DataFrame(data)
medical_wanted_t.columns = medical_wanted_t.iloc[0]
medical_wanted_t = medical_wanted_t[1:]

with open(filepath+csvfilenames[6]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

targetrxdata = pd.DataFrame(data)
targetrxdata.columns = targetrxdata.iloc[0]
targetrxdata = targetrxdata[1:]

In [ ]:
#medical_wanted['STD_COST']=medical_wanted['STD_COST'].apply(pd.to_numeric) #change entries to numbers
#rxdata.loc[rxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
#rxdata['STD_COST']=rxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers
members['year_of_birth']=members['year_of_birth'].apply(pd.to_numeric)
members['totalcost']=members['totalcost'].apply(pd.to_numeric)
members['targetcost']=members['targetcost'].apply(pd.to_numeric)
members['rxcost']=members['rxcost'].apply(pd.to_numeric)
members['targetrxcost']=members['targetrxcost'].apply(pd.to_numeric)
members['medicalcost']=members['medicalcost'].apply(pd.to_numeric)
members['targetmedicalcost']=members['targetmedicalcost'].apply(pd.to_numeric)
members['hospicevisits']=members['hospicevisits'].apply(pd.to_numeric)
members['officevisits']=members['officevisits'].apply(pd.to_numeric)
members['psychvisits']=members['psychvisits'].apply(pd.to_numeric)
members['emervisits']=members['emervisits'].apply(pd.to_numeric)
members['elderlyvisits']=members['elderlyvisits'].apply(pd.to_numeric)
members['unknownvisits']=members['unknownvisits'].apply(pd.to_numeric)
members['homevisits']=members['homevisits'].apply(pd.to_numeric)
members['dialysisvisits']=members['dialysisvisits'].apply(pd.to_numeric)

rxdata['DAYS_FROM_DIAG'] = rxdata['DAYS_FROM_DIAG'].apply(pd.to_numeric)
medical_wanted['DAYS_FROM_DIAG'] = medical_wanted['DAYS_FROM_DIAG'].apply(pd.to_numeric)

#medical_wanted_t['STD_COST']=medical_wanted_t['STD_COST'].apply(pd.to_numeric) #change entries to numbers
#targetrxdata.loc[targetrxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
#targetrxdata['STD_COST']=targetrxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers
#targetrxdata['DAYS_FROM_DIAG'] = targetrxdata['DAYS_FROM_DIAG'].apply(pd.to_numeric)
#medical_wanted_t['DAYS_FROM_DIAG'] = medical_wanted_t['DAYS_FROM_DIAG'].apply(pd.to_numeric)

In [ ]:
#all potentially completely useless
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].min()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'minrxday'] = dummy.iat[k]
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].max()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'maxrxday'] = dummy.iat[k]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].min()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'minmedicalday'] = dummy.iat[k]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].max()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'maxmedicalday'] = dummy.iat[k]

In [ ]:
members['rxfills']=0
dummy = rxdata.groupby('PATID')
patids = list(members.index)

for k in patids:
    try:
        members.loc[k,'rxfills']=len(dummy.get_group(k))
    except KeyError:
        pass

In [ ]:
#Interesting codes in AHFSCLSS (so far) 
#681200: birthcontrol, 
#682004: biguanides (diabetes related), 
#682008: insulin 
#281604: antidepressents


In [ ]:
members['daysperinsulinrx']=0
members['daysperantideprx']=0
members['daysperbcrx']=0
members['daysperbiguaniderx']=0
members['daysperalphaglucosidaserx']=0

patids = list(members.index)
rxdata = rxdata.sort_values('DAYS_FROM_DIAG')
dummy = rxdata.groupby('PATID')
for k in patids:
    try:
        tinydummy=dummy.get_group(k)
        moretinydummy=tinydummy[tinydummy['AHFSCLSS'].str.startswith('682008')]
        if len(moretinydummy)>0:
            members.loc[k,'daysperinsulinrx']=moretinydummy['DAYS_FROM_DIAG'].diff(periods=1).mean()
        moretinydummy=tinydummy[tinydummy['AHFSCLSS'].str.startswith('281604')]
        if len(moretinydummy)>0:
            members.loc[k,'daysperantideprx']=moretinydummy['DAYS_FROM_DIAG'].diff(periods=1).mean()
        moretinydummy=tinydummy[tinydummy['AHFSCLSS'].str.startswith('681200')]
        if len(moretinydummy)>0:
            members.loc[k,'daysperbcrx']=moretinydummy['DAYS_FROM_DIAG'].diff(periods=1).mean()
        moretinydummy=tinydummy[tinydummy['AHFSCLSS'].str.startswith('682004')]
        if len(moretinydummy)>0:
            members.loc[k,'daysperbiguaniderx']=moretinydummy['DAYS_FROM_DIAG'].diff(periods=1).mean()
        moretinydummy=tinydummy[tinydummy['AHFSCLSS'].str.startswith('682002')]
        if len(moretinydummy)>0:
            members.loc[k,'daysperalphaglucosidaserx']=moretinydummy['DAYS_FROM_DIAG'].diff(periods=1).mean()
    except KeyError:
        pass

In [ ]:
members['lasttimerxnodoctor']=members['maxrxday']-members['maxmedicalday']
members['firsttimerxnodoctor']=members['minmedicalday']-members['minrxday']

In [ ]:
#Plot average cost for all POS codes
location_series = medical_wanted.groupby('POS')['STD_COST'].mean()

location_series.plot.bar()
plt.ylim([0,2000])
plt.show()

In [ ]:
members.columns

In [ ]:
# A collection of plots:
#Note: in this situation, we're looking for people who cost around 12k and more:
members.sort_values('targetcost', ascending = 0).iloc[6000]

In [ ]:
# Also note we're looking for a little more than the upper 15% of costs:
series = members['targetcost']
series.quantile([.15, .3, .5, .7, .85, .9 , .95, 1])

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
#colors = {'F': 'red', 'M': 'blue', 'U': 'green'}
members.plot.scatter(x ='totalcost', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
#plt.ylim(0, 300000) #not the actual upper limit, ignoring ncdude
#plt.xlim(0, 3000000)
plt.show()

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
members.plot.scatter(x ='totalcost', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 110000) #not the actual upper limit, ignoring ncdude
plt.xlim(100000, 3000000)
plt.show()
#it appears we can safely exclude people who cost more than 1000000 to begin with.
#Err, include.

In [ ]:
submembers = members[members['totalcost']<1000000]

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='totalcost', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
#plt.ylim(0, 300000) #not the actual upper limit, ignoring ncdude
#plt.xlim(0, 3000000)
plt.show()

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='rxcost', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.show()

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='hospicevisits', y = 'totalcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 600000) #actually upper limit for this subpopulation
plt.xlim(1, 50) #ignore the 0s
plt.show()

In [ ]:
submembers = submembers[submembers['hospicevisits']<15]

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='rxcost', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
#plt.ylim(0, 100000) #not the actual upper limit
plt.xlim(0,1000000)
plt.show()

In [ ]:
#can safely remove people who spent more than 400000 on prescriptions 
#(but what's going on with the people who went from 300000 to 0???)
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='rxcost', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 100000) # not upper limit for this subpopulation
plt.xlim(300000, 800000) #ignore the 0s
plt.show()

In [ ]:
submembers = submembers[submembers['rxcost']<400000]

In [ ]:
#there's a point at which high numbers of office visits imply something
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='officevisits', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 100000) # not upper limit for this subpopulation
#plt.xlim(200, 900) #just compressing a little
plt.show()

In [ ]:
#not so much with high numbers of emergency room visits
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='emervisits', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 100000) # not upper limit for this subpopulation
plt.xlim(50, 300) #just compressing a little
plt.show()

In [ ]:
#maybe with psychiatric though
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='psychvisits', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
#plt.ylim(0, 100000) # not upper limit for this subpopulation
#plt.xlim(20, 60) #just compressing a little
plt.show()

In [ ]:
#=most recent rx - most recent medical visit + least recent office visit - least recent rx
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
members.plot.scatter(x ='lasttimerxnodoctor', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 100000) #not upper limit for this subpopulation
#plt.xlim(-50, 50)
plt.show()

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
members.plot.scatter(x ='daysperinsulinrx', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
#plt.ylim(0, 100000) #not upper limit for this subpopulation
#plt.xlim(2, 100) 
plt.show()

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
members.plot.scatter(x ='daysperantideprx', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 100000) #not upper limit for this subpopulation
plt.xlim(50, 100) 
plt.show()

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
members.plot.scatter(x ='daysperbcrx', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 100000) #not upper limit for this subpopulation
plt.xlim(2,100) 
plt.show()

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
members.plot.scatter(x ='daysperbiguaniderx', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 100000) #not upper limit for this subpopulation
plt.xlim(2, 1200) 
plt.show()

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
members.plot.scatter(x ='daysperalphaglucosidaserx', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 100000) #not upper limit for this subpopulation
plt.xlim(2, 1200) 
plt.show()

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
members.plot.scatter(x ='rxfills', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 1000000) #not upper limit for this subpopulation
#plt.xlim(2, 1200) 
plt.show()

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
members.plot.scatter(x ='rxfills', y = 'rxcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 1000000) #not upper limit for this subpopulation
#plt.xlim(2, 1200) 
plt.show()

In [ ]:
members.columns

In [ ]:
members_for_work = members
dummies = pd.get_dummies(members_for_work.region)
members_for_work = members_for_work.join(dummies)
dummies = pd.get_dummies(members_for_work.gender)
members_for_work = members_for_work.join(dummies)

In [ ]:
members_for_work = members_for_work.drop('gender', axis = 1)
members_for_work = members_for_work.drop('STATE', axis = 1)
members_for_work = members_for_work.drop('region', axis = 1)
members_for_work = members_for_work.drop('minmedicalday', axis = 1)
members_for_work = members_for_work.drop('maxmedicalday', axis = 1)
members_for_work = members_for_work.drop('minrxday', axis = 1)
members_for_work = members_for_work.drop('maxrxday', axis = 1)
members_for_work = members_for_work.drop('targetrxcost', axis = 1)
members_for_work = members_for_work.drop('targetmedicalcost', axis = 1)
members_for_work=members_for_work.fillna(0)

In [ ]:
#the next few cells run a really lousy linear regression.  At least if you're lucky they do.
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
members_train = members_for_work[:30000]
members_test = members_for_work[30000:]
members_y_train = members_train[['targetcost']]
members_x_train = members_train.drop('targetcost', axis=1)
members_y_test = members_test[['targetcost']]
members_x_test = members_test.drop('targetcost', axis=1)

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(members_x_train, members_y_train)
members_y_pred= (regr.predict(members_x_test))

In [ ]:
mean_squared_error(members_y_test, members_y_pred)

In [ ]:
regr.coef_


In [ ]:
members_x_train.columns

In [ ]:
plt.scatter(members_y_test, members_y_pred)
plt.xlim=(0,100000)
plt.show()

In [ ]:
members_for_trees = members_for_work

In [ ]:
members_for_trees_expensive = members_for_work[members_for_work['targetcost']>=20000]
members_for_trees_expensive['expensive']=True
members_for_trees_inexpensive = members_for_work[members_for_work['targetcost']<20000]
members_for_trees_inexpensive['expensive']=False
members_for_trees = members_for_trees_expensive.append(members_for_trees_inexpensive)
msk = np.random.rand(len(members_for_trees)) < 0.8
members_train = members_for_trees[msk]
members_test = members_for_trees[~msk]
print len(members_train)
print len(members_test)

In [ ]:
y_train = members_train['expensive']
y_test = members_test['expensive']
x_train = members_train.drop('expensive', axis = 1)
x_train = x_train.drop('targetcost', axis = 1)
x_test = members_test.drop('expensive', axis = 1)
x_test = x_test.drop('targetcost', axis = 1)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(min_samples_split=2000, random_state=4)
dt.fit(x_train, y_train)
#I played around with samples_split and we're not going to do better than 33% true accuracy, it would seem

In [ ]:
pred = list(dt.predict(x_test))

In [ ]:
members_test['pred']=pred

In [ ]:
#hell if I know how to make a confusion matrix
check=members_test[['targetcost','expensive', 'pred']].sort_values('targetcost', ascending = 0)
check_truetrue = check[check['expensive'] & check['pred']]
check_true = check[check['expensive']]
#check_falsefalse = check[(not check['expensive']) & (not check['pred'])]
#check_falseneg = check[(check['expensive']) & (not check['pred'])]
#check_falsepos = check[(not check['expensive']) & (check['pred'])]

In [ ]:
float(len(check_truetrue))/len(check_true)

In [ ]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print "def tree({}):".format(", ".join(feature_names))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print "{}if {} <= {}:".format(indent, name, threshold)
            recurse(tree_.children_left[node], depth + 1)
            print "{}else:  # if {} > {}".format(indent, name, threshold)
            recurse(tree_.children_right[node], depth + 1)
        else:
            print "{}return {}".format(indent, tree_.value[node])

    recurse(0, 1)

In [ ]:
tree_to_code(dt, x_train.columns)

In [ ]:
y_train = members_for_trees['expensive']
x_train = members_for_trees.drop('expensive', axis = 1)
x_train = x_train.drop('targetcost', axis = 1)

from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(x_train, y_train)
print(model.feature_importances_)

In [ ]:
x_train.columns